키워드:
- 안전 보건 진단, 안전 컨설팅, 위험성 평가, 안전 보건 관리 체계 구축

In [20]:
from dotenv import load_dotenv
import os

load_dotenv()  # 기본적으로 .env 로드

api_base = os.getenv("OPENAPI_PPS_BASE_URL")
api_url = os.getenv("OPENAPI_PPS_SEARCH")
token = os.getenv("OPENAPI_PPS_TOKEN")

print(api_base + api_url) 
print(token)

https://apis.data.go.kr/1230000/ad/BidPublicInfoService/getBidPblancListInfoCnstwkPPSSrch
c4d72abb8738eac53958700e008ad2d7310c5aba26f8c1a050d65d7e2339d5ae


# 입찰공고공사조회

In [60]:
import requests
params = {
    "serviceKey": token,   # 중요
    "pageNo": 1,
    "numOfRows": 50,
    "inqryDiv": 1,
    "inqryBgnDt": "202601020000",
    "inqryEndDt": "202602011200",
    "bidNtceNm": "친환경",
    "bidClseExcpYn": "Y",
    "type": "json",                 # 🔥 _type 아님
}

response = requests.get(api_base + api_url, params=params, timeout=10)

print(response.url)
print(response.status_code)
response.raise_for_status()

data = response.json()
data

https://apis.data.go.kr/1230000/ad/BidPublicInfoService/getBidPblancListInfoCnstwkPPSSrch?serviceKey=c4d72abb8738eac53958700e008ad2d7310c5aba26f8c1a050d65d7e2339d5ae&pageNo=1&numOfRows=50&inqryDiv=1&inqryBgnDt=202601020000&inqryEndDt=202602011200&bidNtceNm=%EC%B9%9C%ED%99%98%EA%B2%BD&bidClseExcpYn=Y&type=json
200


{'response': {'header': {'resultCode': '00', 'resultMsg': '정상'},
  'body': {'items': [{'bidNtceNo': 'R26BK01294518',
     'bidNtceOrd': '000',
     'reNtceYn': 'N',
     'rgstTyNm': '조달청 또는 나라장터 자체 공고건',
     'ntceKindNm': '등록공고',
     'intrbidYn': 'N',
     'bidNtceDt': '2026-01-26 11:30:10',
     'refNo': '1',
     'bidNtceNm': '광역권역별 친환경 농업도시 충전 인프라 조성사업(남부권역) 구축공사',
     'ntceInsttCd': 'ZT04187',
     'ntceInsttNm': '채비(주)',
     'dminsttCd': 'ZT04187',
     'dminsttNm': '채비(주)',
     'bidMethdNm': '전자입찰',
     'cntrctCnclsMthdNm': '제한경쟁',
     'ntceInsttOfclNm': '강동협',
     'ntceInsttOfclTelNo': '***********',
     'ntceInsttOfclEmailAdrs': 'dh.kang2@chaevi.co.kr',
     'exctvNm': '강동협',
     'bidQlfctRgstDt': '2026-02-03 18:00',
     'cmmnSpldmdAgrmntRcptdocMethd': '없음',
     'cmmnSpldmdAgrmntClseDt': '',
     'cmmnSpldmdCorpRgnLmtYn': '',
     'bidBeginDt': '2026-01-26 14:00:00',
     'bidClseDt': '2026-02-04 17:00:00',
     'opengDt': '2026-02-04 18:00:00',
     'ntceSpecDocUrl

# 입찰공고용역조회

In [61]:
import requests
params = {
    "serviceKey": token,   # 중요
    "pageNo": 1,
    "numOfRows": 50,
    "inqryDiv": 1,
    "inqryBgnDt": "202601020000",
    "inqryEndDt": "202602011200",
    "bidNtceNm": "친환경",
    "bidClseExcpYn": "Y",
    "type": "json",                 # 🔥 _type 아님
}
# /getBidPblancListInfoThngPPSSrch
response = requests.get(api_base + "/getBidPblancListInfoServcPPSSrch", params=params, timeout=10)

print(response.url)
print(response.status_code)
response.raise_for_status()

data = response.json()
data

https://apis.data.go.kr/1230000/ad/BidPublicInfoService/getBidPblancListInfoServcPPSSrch?serviceKey=c4d72abb8738eac53958700e008ad2d7310c5aba26f8c1a050d65d7e2339d5ae&pageNo=1&numOfRows=50&inqryDiv=1&inqryBgnDt=202601020000&inqryEndDt=202602011200&bidNtceNm=%EC%B9%9C%ED%99%98%EA%B2%BD&bidClseExcpYn=Y&type=json
200


{'response': {'header': {'resultCode': '00', 'resultMsg': '정상'},
  'body': {'items': [{'bidNtceNo': 'R26BK01293604',
     'bidNtceOrd': '000',
     'reNtceYn': 'N',
     'rgstTyNm': '조달청 또는 나라장터 자체 공고건',
     'ntceKindNm': '등록공고',
     'intrbidYn': 'N',
     'bidNtceDt': '2026-01-26 13:47:53',
     'refNo': '서산시 공고 제2026-280호',
     'bidNtceNm': '2026년 친환경 조사료 생산단지 조성',
     'ntceInsttCd': '4530000',
     'ntceInsttNm': '충청남도 서산시',
     'dminsttCd': '4530000',
     'dminsttNm': '충청남도 서산시',
     'bidMethdNm': '전자입찰',
     'cntrctCnclsMthdNm': '제한경쟁',
     'ntceInsttOfclNm': '박영진',
     'ntceInsttOfclTelNo': '041-660-2272',
     'ntceInsttOfclEmailAdrs': '',
     'exctvNm': '박영진',
     'bidQlfctRgstDt': '2026-02-01 18:00',
     'cmmnSpldmdAgrmntRcptdocMethd': '없음',
     'cmmnSpldmdAgrmntClseDt': '',
     'cmmnSpldmdCorpRgnLmtYn': '',
     'bidBeginDt': '2026-01-29 10:00:00',
     'bidClseDt': '2026-02-02 10:00:00',
     'opengDt': '2026-02-02 11:00:00',
     'ntceSpecDocUrl1': 'https://w